# 识别数字
欢迎大家来到这次实验，在这次实验中我们将使用PaddlePaddle来实现三种不同的分类器，用于识别手写数字。三种分类器所基于的模型分别为Softmax回归、多层感知器、卷积神经网络。现在让我们进入实验来看看基于不同模型的分类器的差异吧!

** 你将学会 **

- 实现一个基于Softmax回归的分类器，用于识别手写数字

- 实现一个基于多层感激器的分类器，用于识别手写数字

- 实现一个基于卷积神经网络的分类器，用于识别手写数字

- 卷积神经网络的基本组成和搭建

现在让我们进入实验吧！

## 1 - 引用库

首先，载入几个需要用到的库，它们分别是：

- numpy：一个python的基本库，用于科学计算
- matplotlib：用于生成图，在验证模型准确率和展示成本变化趋势时会使用到
- paddle.v2：PaddlePaddle深度学习框架
- Ploter：PaddlePaddle深度学习框架体提供的绘图工具
- os：在本例中用于获取文件或目录的路径
- Image：用于处理图像数据

此外，定义一些全局变量，其中`WITH_GPU`表示是否使用GPU，`STEP`等其余变量用于后续绘制曲线图。

In [1]:
import os

import matplotlib
import numpy as np
import paddle.v2 as paddle
from paddle.v2.plot import Ploter
from PIL import Image


WITH_GPU = os.getenv('WITH_GPU', '0') != '0'

# 绘图相关
STEP = 0

TRAIN_TITLE_COST = "Train cost"
TEST_TITLE_COST = "Test cost"

TRAIN_TITLE_ERROR = "Train error rate"
TEST_TITLE_ERROR = "Test error rate"

** 问题描述： **

当我们学习编程的时候，编写的第一个程序一般是实现打印"Hello World"。而机器学习（或深度学习）的入门教程，一般都是 [MNIST](http://yann.lecun.com/exdb/mnist/) 数据库上的手写识别问题。原因是手写识别属于典型的图像分类问题，比较简单，同时MNIST数据集也很完备。

** 你的目标： **

构建三种不同的分类器来对手写数字进行识别

** 数据集分析： **

MNIST数据集作为一个简单的计算机视觉数据集，包含一系列如图1所示的手写数字图片和对应的标签。图片是28x28的像素矩阵，标签则对应着0~9的10个数字。每张图片都经过了大小归一化和居中处理。

<p align="center">
<img src="image/mnist_example_image.png" width="400"><br/>
<center>图1. MNIST图片示例</center>
</p>

MNIST数据集是从 [NIST](https://www.nist.gov/srd/nist-special-database-19) 的Special Database 3（SD-3）和Special Database 1（SD-1）构建而来。由于SD-3是由美国人口调查局的员工进行标注，SD-1是由美国高中生进行标注，因此SD-3比SD-1更干净也更容易识别。Yann LeCun等人从SD-1和SD-3中各取一半作为MNIST的训练集（60000条数据）和测试集（10000条数据），其中训练集来自250位不同的标注员，此外还保证了训练集和测试集的标注员是不完全相同的。

Yann LeCun早先在手写字符识别上做了很多研究，并在研究过程中提出了卷积神经网络（Convolutional Neural Network），大幅度地提高了手写字符的识别能力，也因此成为了深度学习领域的奠基人之一。如今的深度学习领域，卷积神经网络占据了至关重要的地位，从最早Yann LeCun提出的简单LeNet，到如今ImageNet大赛上的优胜模型VGGNet、GoogLeNet、ResNet等（请参见[图像分类](https://github.com/PaddlePaddle/book/tree/develop/03.image_classification) 教程），人们在图像分类领域，利用卷积神经网络得到了一系列惊人的结果。

有很多算法在MNIST上进行实验。1998年，LeCun分别用单层线性分类器、多层感知器（Multilayer Perceptron, MLP）和多层卷积神经网络LeNet进行实验，使得测试集上的误差不断下降（从12%下降到0.7%）\[[1](#参考文献)\]。此后，科学家们又基于K近邻（K-Nearest Neighbors）算法\[[2](#参考文献)\]、支持向量机（SVM）\[[3](#参考文献)\]、神经网络\[[4-7](#参考文献)\]和Boosting方法\[[8](#参考文献)\]等做了大量实验，并采用多种预处理方法（如去除歪曲、去噪、模糊等）来提高识别的准确率。

本教程中，我们从简单的模型Softmax回归开始，带大家入门手写字符识别，并逐步进行模型优化。

输入值：

- $X$是输入：MNIST图片是$28\times28$ 的二维图像，为了进行计算，我们将其转化为$784$维向量，即$X=\left ( x_0, x_1, \dots, x_{783} \right )$。转化的具体做法：每张图片是由$28\times28$个像素构成的，将其按固定顺序（如按行或者按列）展开成为一个行向量，并将每个原始像素值归一化为$\left[0,1\right]$之间的数值。

- $L$是图片的真实标签：$L=\left ( l_0, l_1, \dots, l_9 \right )$也是10维，但只有一维为1，其他都为0。值为1的维度对应图片表示的真实数字，例如$L=\left ( 1, 0, \dots, 0 \right )$表示图片对应的数字是1。

输出值：

- $Y$是输出：分类器的输出是10类数字（0-9），即$Y=\left ( y_0, y_1, \dots, y_9 \right )$，每一维$y_i$代表图片分类为第$i$类数字的概率。

## 2 - 数据获取

** 文件路径 **

PaddlePaddle在API中提供了自动加载[MNIST](http://yann.lecun.com/exdb/mnist/)数据的模块`paddle.dataset.mnist`。

加载后的数据位于`/home/username/.cache/paddle/dataset/mnist`下：


|    文件名称          |       说明              |
|----------------------|-------------------------|
|train-images-idx3-ubyte|  训练数据图片，60,000条数据 |
|train-labels-idx1-ubyte|  训练数据标签，60,000条数据 |
|t10k-images-idx3-ubyte |  测试数据图片，10,000条数据 |
|t10k-labels-idx1-ubyte |  测试数据标签，10,000条数据 |

## 3 - 知识点介绍

### 3.1 常见激活函数介绍  

- sigmoid激活函数： $ f(x) = sigmoid(x) = \frac{1}{1+e^{-x}} $

- tanh激活函数： $ f(x) = tanh(x) = \frac{e^x-e^{-x}}{e^x+e^{-x}} $

  实际上，tanh函数只是规模变化的sigmoid函数，将sigmoid函数值放大2倍之后再向下平移1个单位：tanh(x) = 2sigmoid(2x) - 1 。

- ReLU激活函数： $ f(x) = max(0, x) $

更详细的介绍请参考[维基百科激活函数](https://en.wikipedia.org/wiki/Activation_function)。

### 3.2 - Softmax回归(Softmax Regression)

最简单的Softmax回归模型是先将输入层经过一个全连接层得到的特征，然后直接通过softmax 函数进行多分类\[[9](#参考文献)\]。

输入层的数据$X$传到输出层，在激活操作之前，会乘以相应的权重 $W$ ，并加上偏置变量 $b$ ，具体如下：

$$ y_i = \text{softmax}(\sum_j W_{i,j}x_j + b_i) $$

其中 $ \text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}} $

对于有 $N$ 个类别的多分类问题，指定 $N$ 个输出节点，$N$ 维结果向量经过softmax将归一化为 $N$ 个[0,1]范围内的实数值，分别表示该样本属于这 $N$ 个类别的概率。此处的 $y_i$ 即对应该图片为数字 $i$ 的预测概率。

在分类问题中，我们一般采用交叉熵代价损失函数（cross entropy），公式如下：

$$  \text{crossentropy}(label, y) = -\sum_i label_ilog(y_i) $$

图2为softmax回归的网络图，图中权重用蓝线表示、偏置用红线表示、+1代表偏置参数的系数为1。

<p align="center">
<img src="image/softmax_regression.png" width=400><br/>
<center>图2. softmax回归网络结构图</center><br/>
</p>

### 3.3 - 多层感知器(Multilayer Perceptron, MLP)

Softmax回归模型采用了最简单的两层神经网络，即只有输入层和输出层，因此其拟合能力有限。为了达到更好的识别效果，我们考虑在输入层和输出层中间加上若干个隐藏层\[[10](#参考文献)\]。

1.  经过第一个隐藏层，可以得到 $ H_1 = \phi(W_1X + b_1) $，其中$\phi$代表激活函数，常见的有sigmoid、tanh或ReLU等函数。
2.  经过第二个隐藏层，可以得到 $ H_2 = \phi(W_2H_1 + b_2) $。
3.  最后，再经过输出层，得到的$Y=\text{softmax}(W_3H_2 + b_3)$，即为最后的分类结果向量。


图3为多层感知器的网络结构图，图中权重用蓝线表示、偏置用红线表示、+1代表偏置参数的系数为1。

<p align="center">
<img src="image/mlp.png" width=500><br/>
<center>图3. 多层感知器网络结构图<br/></center>
</p>

### 3.4 - 卷积神经网络(Convolutional Neural Network, CNN)

在多层感知器模型中，将图像展开成一维向量输入到网络中，忽略了图像的位置和结构信息，而卷积神经网络能够更好的利用图像的结构信息。[LeNet-5](http://yann.lecun.com/exdb/lenet/)是一个较简单的卷积神经网络。图4显示了其结构：输入的二维图像，先经过两次卷积层到池化层，再经过全连接层，最后使用softmax分类作为输出层。下面我们主要介绍卷积层和池化层。

<p align="center">
<img src="image/cnn.png"><br/>
<center> 图4. LeNet-5卷积神经网络结构<br/></center>
</p>

#### 卷积层

卷积层是卷积神经网络的核心基石。在图像识别里我们提到的卷积是二维卷积，即离散二维滤波器（也称作卷积核）与二维图像做卷积操作，简单的讲是二维滤波器滑动到二维图像上所有位置，并在每个位置上与该像素点及其领域像素点做内积。卷积操作被广泛应用与图像处理领域，不同卷积核可以提取不同的特征，例如边沿、线性、角等特征。在深层卷积神经网络中，通过卷积操作可以提取出图像低级到复杂的特征。

<p align="center">
<img src="image/conv_layer.png" width='750'><br/>
<center>图5. 卷积层图片<br/></center>
</p>

图5给出一个卷积计算过程的示例图，输入图像大小为$H=5,W=5,D=3$，即$5 \times 5$大小的3通道（RGB，也称作深度）彩色图像。这个示例图中包含两（用$K$表示）组卷积核，即图中滤波器$W_0$和$W_1$。在卷积计算中，通常对不同的输入通道采用不同的卷积核，如图示例中每组卷积核包含（$D=3）$个$3 \times 3$（用$F \times F$表示）大小的卷积核。另外，这个示例中卷积核在图像的水平方向（$W$方向）和垂直方向（$H$方向）的滑动步长为2（用$S$表示）；对输入图像周围各填充1（用$P$表示）个0，即图中输入层原始数据为蓝色部分，灰色部分是进行了大小为1的扩展，用0来进行扩展。经过卷积操作得到输出为$3 \times 3 \times 2$（用$H_{o} \times W_{o} \times K$表示）大小的特征图，即$3 \times 3$大小的2通道特征图，其中$H_o$计算公式为：$H_o = (H - F + 2 \times P)/S + 1$，$W_o$同理。 而输出特征图中的每个像素，是每组滤波器与输入图像每个特征图的内积再求和，再加上偏置$b_o$，偏置通常对于每个输出特征图是共享的。输出特征图$o[:,:,0]$中的最后一个$-2$计算如图5右下角公式所示。

在卷积操作中卷积核是可学习的参数，经过上面示例介绍，每层卷积的参数大小为$D \times F \times F \times K$。在多层感知器模型中，神经元通常是全部连接，参数较多。而卷积层的参数较少，这也是由卷积层的主要特性即局部连接和共享权重所决定。

- 局部连接：每个神经元仅与输入神经元的一块区域连接，这块局部区域称作感受野（receptive field）。在图像卷积操作中，即神经元在空间维度（spatial dimension，即上图示例H和W所在的平面）是局部连接，但在深度上是全部连接。对于二维图像本身而言，也是局部像素关联较强。这种局部连接保证了学习后的过滤器能够对于局部的输入特征有最强的响应。局部连接的思想，也是受启发于生物学里面的视觉系统结构，视觉皮层的神经元就是局部接受信息的。

- 权重共享：计算同一个深度切片的神经元时采用的滤波器是共享的。例如图4中计算$o[:,:,0]$的每个每个神经元的滤波器均相同，都为$W_0$，这样可以很大程度上减少参数。共享权重在一定程度上讲是有意义的，例如图片的底层边缘特征与特征在图中的具体位置无关。但是在一些场景中是无意的，比如输入的图片是人脸，眼睛和头发位于不同的位置，希望在不同的位置学到不同的特征 (参考[斯坦福大学公开课]( http://cs231n.github.io/convolutional-networks/))。请注意权重只是对于同一深度切片的神经元是共享的，在卷积层，通常采用多组卷积核提取不同特征，即对应不同深度切片的特征，不同深度切片的神经元权重是不共享。另外，偏重对同一深度切片的所有神经元都是共享的。

通过介绍卷积计算过程及其特性，可以看出卷积是线性操作，并具有平移不变性（shift-invariant），平移不变性即在图像每个位置执行相同的操作。卷积层的局部连接和权重共享使得需要学习的参数大大减小，这样也有利于训练较大卷积神经网络。

#### 池化层

<p align="center">
<img src="image/max_pooling.png" width="400px"><br/>
<center>图6. 池化层图片<br/></center>
</p>

一般情况下，在连续的卷积层之间会周期性地插入一个池化层（也称汇聚层），其处理输入数据的准则被称为池化函数。池化函数在计算某一位置的输出时，会计算该位置相邻区域的输出的某种总体统计特征，作为网络在该位置的输出。池化层的作用是逐渐降低数据体的空间尺寸，从而减少网络中参数的数量以及耗费的计算资源，同时也能有效控制过拟合。

池化包括最大池化、平均池化、L-2范数池化等。以最大池化(Max Pooling)为例，池化层使用最大化(Max)操作，即用一定区域内输入的最大值作为该区域的输出。最大池化最常用的形式是使用尺寸为$2\times2$的滤波器、步长为2来对每个深度切片进行降采样，每个Max操作是从4个数字中取最大值（也就是在深度切片中某个的区域），这样可以将其中75%的激活信息都过滤掉，而保持数据体通道数不变,具体示例如图6所示。

更详细的关于卷积神经网络的具体知识可以参考[斯坦福大学公开课]( http://cs231n.github.io/convolutional-networks/ )和[图像分类](https://github.com/PaddlePaddle/book/blob/develop/image_classification/README.md)教程。


## 4 - 构建分类器

我们利用PaddlePaddle构建三个不同的分类器：

** Softmax回归：**

只通过一层简单的以Softmax为激活函数的全连接层得到分类结果。具体过程和网络结构如图7所示：784维的输入特征经过节点数目为10的全连接层后，直接通过Softmax函数进行多分类。

<p align="center">
<img src="image/softmax_book.png" width="350px"><br/>
<center>图7. 基于Softmax回归的分类器<br/></center>
</p>

** 练习 **

按照上述描述实现Softmax回归分类器对应的softmax_regression(img)函数。

In [2]:
def softmax_regression(img):
    """
    定义softmax分类器：
        只通过一层简单的以softmax为激活函数的全连接层，可以得到分类的结果
    Args:
        img -- 输入的原始图像数据
    Return:
        predict_image -- 分类的结果
    """
    ### 练习代码开始处 ###(1-3行代码)
    predict = paddle.layer.fc(input=img,
                              size=10,
                              act=paddle.activation.Softmax())
    ### 练习代码结束处 ###
    return predict


** 多层感知器：**

下面代码实现了一个含有两个隐藏层（即全连接层）的多层感知器。其中两个隐藏层的激活函数均采用ReLU，输出层的激活函数用Softmax。Softmax回归模型采用了最简单的两层神经网络，即只有输入层和输出层，因此其拟合能力有限。为了达到更好的识别效果，我们考虑在输入层和输出层中间加上若干个隐藏层,从而得到了多层感知器模型。其对应的网络结构如图8所示：784维的输入特征，先后经过两个节点数为128和64的全连接层，最后通过Softmax函数进行多分类。

<p align="center">
<img src="image/mlp_book.png" width="450px"><br/>
<center>图7. 基于多层感知器模型的分类器<br/></center>
</p>

** 练习 **
按照上述描述定义多层感知器模型对应的函数multilayer_perceptron(img)。

In [3]:
def multilayer_perceptron(img):
    """
    定义多层感知机分类器：
        含有两个隐藏层（即全连接层）的多层感知器
        其中两个隐藏层的激活函数均采用ReLU，输出层的激活函数用Softmax
    Args:
        img -- 输入的原始图像数据
    Return:
        predict_image -- 分类的结果
    """
    # 第一个全连接层，激活函数为ReLU
    ### 练习代码开始处 ### (1行代码)
    hidden1 = paddle.layer.fc(input=img, size=128, act=paddle.activation.Relu())
    ### 练习代码结束处 ###
    
    # 第二个全连接层，激活函数为ReLU
    ### 练习代码开始处 ###（1-3行代码）
    hidden2 = paddle.layer.fc(input=hidden1,
                              size=64,
                              act=paddle.activation.Relu())
    ### 练习代码结束处 ###
    
    # 以softmax为激活函数的全连接输出层，输出层的大小必须为10,对应0-9这10个数字
    ### 练习代码开始处 ###（1-3行代码）
    predict = paddle.layer.fc(input=hidden2,
                              size=10,
                              act=paddle.activation.Softmax())
    ### 练习代码结束处 ###
    return predict


** 卷积神经网络: **

下方代码实现了一个卷积神经网络分类器，其网络结构如图8所示：输入的二维图像，经过两次卷积层后接池化层的结构，在通过输出节点数目为10的以Softmax函数作为激活函数的全连接层后得到多分类输出。

<p align="center">
<img src="image/cnn_book.png" width="700px"><br/>
<center>图8. 卷积神经网络分类器<br/></center>
</p>

** 练习 **

根据上述描述和图示，实现卷积神经网络模型对应的函数convolutional_neural_network(img)。

In [1]:
def convolutional_neural_network(img):
    """
    定义卷积神经网络分类器：
        输入的二维图像，经过两个卷积-池化层，使用以softmax为激活函数的全连接层作为输出层
    Args:
        img -- 输入的原始图像数据
    Return:
        predict -- 分类的结果
    """
    # 第一个卷积-池化层
    ### 练习代码开始处 ### （1-8行代码）
    conv_pool_1 = paddle.networks.simple_img_conv_pool(
        input=img,
        filter_size=5,
        num_filters=20,
        num_channel=1,
        pool_size=2,
        pool_stride=2,
        act=paddle.activation.Relu())
    ### 练习代码结束处 ###
    
    # 第二个卷积-池化层
    ### 练习代码开始处 ### （1-8行代码）
    conv_pool_2 = paddle.networks.simple_img_conv_pool(
        input=conv_pool_1,
        filter_size=5,
        num_filters=50,
        num_channel=20,
        pool_size=2,
        pool_stride=2,
        act=paddle.activation.Relu())
    ### 练习代码结束处 ###
    
    # 以softmax为激活函数的全连接输出层，输出层的大小必须为10,对应0-9这10个数字
    ### 练习代码开始处 ### （1-3行代码）
    predict = paddle.layer.fc(input=conv_pool_2,
                              size=10,
                              act=paddle.activation.Softmax())
    ### 练习代码结束处 ###
    return predict



## 5 - 训练过程

在构建完成分类器后，接下来进入模型的训练过程，关键步骤如下：

- 初始化

- 配置网络结构和设置参数
    - 配置网络结构
    - 定义损失函数cost
    - 创建parameters
    - 定义优化器optimizer

- 模型训练

- 模型检验

- 预测

** (1) 初始化 **

首先进行最基本的初始化操作，在PaddlePaddle中使用paddle.init(use_gpu=False, trainer_count=1)来进行初始化：

- use_gpu=False表示不使用gpu进行训练

- trainer_count=1表示仅使用一个训练器进行训练

In [5]:
# 初始化，设置是否使用gpu，trainer数量
paddle.init(use_gpu=WITH_GPU, trainer_count=1)

定义并调用plot_init()函数用于初始化绘图相关设置。

In [6]:
def plot_init():
    """
    绘图初始化函数：
        初始化绘图相关变量
    Args:
    Return:
        cost_ploter -- 用于绘制cost曲线的变量
        error_ploter -- 用于绘制error_rate曲线的变量
    """
    # 绘制cost曲线所做的初始化设置
    cost_ploter = Ploter(TRAIN_TITLE_COST, TEST_TITLE_COST)

    # 绘制error_rate曲线所做的初始化设置
    error_ploter = Ploter(TRAIN_TITLE_ERROR, TEST_TITLE_ERROR)

    ploter = [cost_ploter, error_ploter]

    return ploter


# 初始化绘图变量
cost_ploter, error_ploter = plot_init()

** (2) 配置网络结构和设置参数 **

接下来需要配置分类网络的结构，首先通过layer.data配置数据输入层，然后配置分类器（代码中提供了三个不同的分类器，每次使用选择其中一个，注释掉其余两个即可），例如下方代码中选用的是卷积神经网络分类器。然后设置损失函数，对于分类问题常常选择交叉熵损失函数。进一步创建参数parameters，定义优化器optimizer。我们将上述过程封装成一个函数，具体如下所示。

In [7]:
def netconfig():
    """
    配置网络结构
    Args:
    Return:
        images -- 输入层
        label -- 标签数据
        predict -- 输出层
        cost -- 损失函数
        parameters -- 模型参数
        optimizer -- 优化器
    """
    
    """
    输入层:
        paddle.layer.data表示数据层,
        name=’pixel’：名称为pixel,对应输入图片特征
        type=paddle.data_type.dense_vector(784)：数据类型为784维(输入图片的尺寸为28*28)稠密向量
    """
    images = paddle.layer.data(
        name='pixel', type=paddle.data_type.dense_vector(784))
        
    """
    数据层:
        paddle.layer.data表示数据层,
        name=’label’：名称为label,对应输入图片的类别标签
        type=paddle.data_type.dense_vector(10)：数据类型为10维(对应0-9这10个数字)稠密向量
    """
    label = paddle.layer.data(
        name='label', type=paddle.data_type.integer_value(10))
        
    """ 
    选择分类器：
        在此之前已经定义了3种不同的分类器，在下面的代码中,
        我们可以通过保留某种方法的调用语句、注释掉其余两种，以选择特定的分类器
    """
    # predict = softmax_regression(images)
    # predict = multilayer_perceptron(images)
    predict = convolutional_neural_network(images)

    # 定义成本函数，addle.layer.classification_cost()函数内部采用的是交叉熵损失函数
    cost = paddle.layer.classification_cost(input=predict, label=label)

    # 利用cost创建参数parameters
    parameters = paddle.parameters.create(cost)
      
    # 创建优化器optimizer，下面列举了2种常用的优化器，不同类型优化器选一即可
    # 创建Momentum优化器，并设置学习率(learning_rate)、动量(momentum)和正则化项(regularization)
    optimizer = paddle.optimizer.Momentum(
        learning_rate=0.01 / 128.0,
        momentum=0.9,
        regularization=paddle.optimizer.L2Regularization(rate=0.0005 * 128))
    
    # 创建Adam优化器，并设置参数beta1、beta2、epsilon
    # optimizer = paddle.optimizer.Adam(beta1=0.9, beta2=0.99, epsilon=1e-06)
    
    config_data = [images, label, predict, cost, parameters, optimizer]
    
    return config_data



** 优化器相关参数说明 **

- 训练方法（optimizer)： 代表训练过程在更新权重时采用动量优化器 `Momentum` ，其中参数0.9代表动量优化每次保持前一次速度的0.9倍。
- 训练速度（learning_rate）： 迭代的速度，与网络的训练收敛速度有关系。
- 正则化（regularization）： 是防止网络过拟合的一种手段，此处采用L2正则化。

我们现在可以调用封装好的netconfig()函数来完成网络结构的配置和参数的设置。

In [8]:
# 定义神经网络结构
images, label, predict, cost, parameters, optimizer = netconfig()

[INFO 2018-03-25 11:05:08,256 layers.py:2563] output for __conv_pool_0___conv: c = 20, h = 24, w = 24, size = 11520
[INFO 2018-03-25 11:05:08,258 layers.py:2691] output for __conv_pool_0___pool: c = 20, h = 12, w = 12, size = 2880
[INFO 2018-03-25 11:05:08,260 layers.py:2563] output for __conv_pool_1___conv: c = 50, h = 8, w = 8, size = 3200
[INFO 2018-03-25 11:05:08,261 layers.py:2691] output for __conv_pool_1___pool: c = 50, h = 4, w = 4, size = 800


*** 事件处理函数 ***

为了将训练过程可视化，我们定义event_handler_plot(event)函数用于输出和绘制训练过程中损失函数和错误率的变化。

In [9]:
# lists用于存储训练的中间结果，包括cost和error_rate信息，初始化为空
lists = []
def event_handler_plot(event):
    """
    定义event_handler_plot事件处理函数：
        事件处理器，可以根据训练过程的信息做相应操作：包括绘图和输出训练结果信息
    Args:
        event -- 事件对象，包含event.pass_id, event.batch_id, event.cost等信息
    Return:
    """
    global STEP
    if isinstance(event, paddle.event.EndIteration):
        # 每训练100次（即100个batch），添加一个绘图点
        if STEP % 100 == 0:
            cost_ploter.append(TRAIN_TITLE_COST, STEP, event.cost)
            # 绘制cost图像，保存图像为‘train_test_cost.png’
            cost_ploter.plot('./train_test_cost')
            error_ploter.append(
                TRAIN_TITLE_ERROR, STEP,
                event.metrics['classification_error_evaluator'])
            # 绘制error_rate图像，保存图像为‘train_test_error_rate.png’
            error_ploter.plot('./train_test_error_rate')
        STEP += 1
        # 每训练100个batch，输出一次训练结果信息
        if event.batch_id % 100 == 0:
            print "Pass %d, Batch %d, Cost %f, %s" % (
                event.pass_id, event.batch_id, event.cost, event.metrics)
    if isinstance(event, paddle.event.EndPass):
        # 保存参数至文件
        with open('params_pass_%d.tar' % event.pass_id, 'w') as param_f:
            trainer.save_parameter_to_tar(param_f)
        # 利用测试数据进行测试
        result = trainer.test(reader=paddle.batch(
            paddle.dataset.mnist.test(), batch_size=128))
        print "Test with Pass %d, Cost %f, %s\n" % (
            event.pass_id, result.cost, result.metrics)
        # 添加测试数据的cost和error_rate绘图数据
        cost_ploter.append(TEST_TITLE_COST, STEP, result.cost)
        error_ploter.append(
            TEST_TITLE_ERROR, STEP,
            result.metrics['classification_error_evaluator'])
        # 存储测试数据的cost和error_rate数据
        lists.append((
            event.pass_id, result.cost,
            result.metrics['classification_error_evaluator']))

**（3）模型训练 **

在完成前述准备工作后，我们可以开始模型的训练。首先构造训练器trainer,然后调用trainer.train函数进行训练，对train函数的参数说明如下：

- paddle.reader.shuffle(paddle.dataset.mnist.train(), buf_size=8192)：表示trainer从paddle.dataset.mnist.train()这个reader中读取了buf_size=8192大小的数据并打乱顺序
- paddle.batch(reader(), batch_size=128)：表示从打乱的数据中再取出batch_size=128大小的数据进行一次迭代训练
- event_handler：事件处理函数，可以自定义event_handler，根据事件信息做相应的操作，下方代码中选择的是event_handler_plot函数
- num_passes：定义训练的迭代次数

In [10]:
# 构造trainer,配置三个参数cost、parameters、update_equation，它们分别表示成本函数、参数和更新公式
trainer = paddle.trainer.SGD(
    cost=cost, parameters=parameters, update_equation=optimizer)

trainer.train(
    reader=paddle.batch(
        paddle.reader.shuffle(paddle.dataset.mnist.train(), buf_size=8192),
        batch_size=128),
    event_handler=event_handler_plot,
    num_passes=10)

Pass 0, Batch 0, Cost 2.976693, {'classification_error_evaluator': 0.875}
Pass 0, Batch 100, Cost 0.303500, {'classification_error_evaluator': 0.0859375}
Pass 0, Batch 200, Cost 0.130422, {'classification_error_evaluator': 0.0390625}
Pass 0, Batch 300, Cost 0.063400, {'classification_error_evaluator': 0.015625}
Pass 0, Batch 400, Cost 0.179245, {'classification_error_evaluator': 0.046875}
Test with Pass 0, Cost 0.063572, {'classification_error_evaluator': 0.021199999377131462}

Pass 1, Batch 0, Cost 0.061532, {'classification_error_evaluator': 0.03125}
Pass 1, Batch 100, Cost 0.062394, {'classification_error_evaluator': 0.0234375}
Pass 1, Batch 200, Cost 0.060313, {'classification_error_evaluator': 0.015625}
Pass 1, Batch 300, Cost 0.096427, {'classification_error_evaluator': 0.0390625}
Pass 1, Batch 400, Cost 0.056513, {'classification_error_evaluator': 0.0234375}
Test with Pass 1, Cost 0.053012, {'classification_error_evaluator': 0.01679999940097332}

Pass 2, Batch 0, Cost 0.034559, 

训练过程是完全自动的，event_handler_plot打印的日志类似如下所示：

```
Pass 0, Batch 0, Cost 3.114700, {'classification_error_evaluator': 0.890625}
Pass 0, Batch 100, Cost 0.127103, {'classification_error_evaluator': 0.046875}
Pass 0, Batch 200, Cost 0.067737, {'classification_error_evaluator': 0.015625}
Pass 0, Batch 300, Cost 0.124578, {'classification_error_evaluator': 0.0390625}
Pass 0, Batch 400, Cost 0.082452, {'classification_error_evaluator': 0.015625}
Test with Pass 0, Cost 0.062783, {'classification_error_evaluator': 0.019700000062584877}

Pass 1, Batch 0, Cost 0.089441, {'classification_error_evaluator': 0.03125}
Pass 1, Batch 100, Cost 0.056283, {'classification_error_evaluator': 0.015625}
Pass 1, Batch 200, Cost 0.062043, {'classification_error_evaluator': 0.0234375}
Pass 1, Batch 300, Cost 0.027225, {'classification_error_evaluator': 0.0078125}
Pass 1, Batch 400, Cost 0.108456, {'classification_error_evaluator': 0.03125}
Test with Pass 1, Cost 0.047607, {'classification_error_evaluator': 0.01549999974668026}
```

训练过程中的cost和error_rate变化曲线如下图所示，可以查看保存在本地的图像文件`train_test_cost.png`和`train_test_error_rate.png`。

<p align="center">
<img src="./train_test_cost.png" width="400px"><br/>
<center>图9. cost变化曲线<br/></center>
</p>

<p align="center">
<img src="./train_test_error_rate.png" width="400px"><br/>
<center>图10. error_rate变化曲线<br/></center>
</p>

在多次迭代中，找到在测试数据上表现最好的一组参数，并输出相应信息：

In [11]:
best = sorted(lists, key=lambda list: float(list[1]))[0]
print 'Best pass is %s, testing Avgcost is %s' % (best[0], best[1])
print 'The classification accuracy is %.2f%%' % (100 - float(best[2]) * 100)

Best pass is 9, testing Avgcost is 0.0309882794308
The classification accuracy is 98.93%


训练之后，检查模型的预测准确度。用MNIST训练的时候，一般Softmax回归模型的分类准确率为约为92.34%，多层感知器为97.66%，卷积神经网络可以达到99.20%。

** (5) 模型检验 **

模型训练完成后，接下来检验模型的准确率。
首先定义一个用来处理输入的图片的函数load_image(file)。它会将输入的图片处理成满足分类器输入要求的格式。

In [28]:
def load_image(file):
    """
    定义读取输入图片的函数：
        读取指定路径下的图片，将其处理成分类网络输入数据对应形式的数据，如数据维度等
    Args:
        file -- 输入图片的文件路径
    Return:
        im -- 分类网络输入数据对应形式的数据
    """
    im = Image.open(file).convert('L')
    im = im.resize((28, 28), Image.ANTIALIAS)
    im = np.array(im).astype(np.float32).flatten()
    im = im / 255.0
    return im

然后我们再定义一个infer(predict, parameters, file)函数，用于预测输入图片的标签。其内部会调用之前定义的load_image(file)函数处理要预测的图片,然后会调用paddle.infe接口来预测图片类型。

In [33]:
def infer(predict, parameters, file):
    """
    定义判断输入图片类别的函数：
        读取并处理指定路径下的图片，然后调用训练得到的模型进行类别预测
    Args:
        predict -- 输出层
        parameters -- 模型参数
        file -- 输入图片的文件路径
    Return:
    """
    # 读取并预处理要预测的图片
    test_data = []
    cur_dir = os.path.dirname('./')
    test_data.append((load_image(cur_dir + file), ))
    
    # 利用训练好的分类模型，对输入的图片类别进行预测
    probs = paddle.infer(
        output_layer=predict, parameters=parameters, input=test_data)
    lab = np.argsort(-probs)
    print "Label of image/infer_3.png is: %d" % lab[0][0]

下面就让我们调用定义好的infer函数来对数字图片的标签进行预测。

In [34]:
# 预测输入图片的类型
infer(predict, parameters, '/image/infer_3.png')

Label of image/infer_3.png is: 3



## 6 - 总结

通过这个练习我们应该记住：

1. 基于Softmax回归、多层感知器、卷积神经网络构建分类器的操作

2. 如何利用PaddlePaddle的API接口对图片进行特征提取和分类


本教程的softmax回归、多层感知器和卷积神经网络是最基础的深度学习模型，后续章节中复杂的神经网络都是从它们衍生出来的，因此这几个模型对之后的学习大有裨益。同时，我们也观察到从最简单的softmax回归变换到稍复杂的卷积神经网络的时候，MNIST数据集上的识别准确率有了大幅度的提升，原因是卷积层具有局部连接和共享权重的特性。在之后学习新模型的时候，希望大家也要深入到新模型相比原模型带来效果提升的关键之处。作为扩展，大家可以用自己的数据，定义自己的网络模型，并完成自己的训练和预测任务。


## 参考文献

1. LeCun, Yann, Léon Bottou, Yoshua Bengio, and Patrick Haffner. ["Gradient-based learning applied to document recognition."](http://ieeexplore.ieee.org/abstract/document/726791/) Proceedings of the IEEE 86, no. 11 (1998): 2278-2324.
2. Wejéus, Samuel. ["A Neural Network Approach to Arbitrary SymbolRecognition on Modern Smartphones."](http://www.diva-portal.org/smash/record.jsf?pid=diva2%3A753279&dswid=-434) (2014).
3. Decoste, Dennis, and Bernhard Schölkopf. ["Training invariant support vector machines."](http://link.springer.com/article/10.1023/A:1012454411458) Machine learning 46, no. 1-3 (2002): 161-190.
4. Simard, Patrice Y., David Steinkraus, and John C. Platt. ["Best Practices for Convolutional Neural Networks Applied to Visual Document Analysis."](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.160.8494&rep=rep1&type=pdf) In ICDAR, vol. 3, pp. 958-962. 2003.
5. Salakhutdinov, Ruslan, and Geoffrey E. Hinton. ["Learning a Nonlinear Embedding by Preserving Class Neighbourhood Structure."](http://www.jmlr.org/proceedings/papers/v2/salakhutdinov07a/salakhutdinov07a.pdf) In AISTATS, vol. 11. 2007.
6. Cireşan, Dan Claudiu, Ueli Meier, Luca Maria Gambardella, and Jürgen Schmidhuber. ["Deep, big, simple neural nets for handwritten digit recognition."](http://www.mitpressjournals.org/doi/abs/10.1162/NECO_a_00052) Neural computation 22, no. 12 (2010): 3207-3220.
7. Deng, Li, Michael L. Seltzer, Dong Yu, Alex Acero, Abdel-rahman Mohamed, and Geoffrey E. Hinton. ["Binary coding of speech spectrograms using a deep auto-encoder."](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.185.1908&rep=rep1&type=pdf) In Interspeech, pp. 1692-1695. 2010.
8. Kégl, Balázs, and Róbert Busa-Fekete. ["Boosting products of base classifiers."](http://dl.acm.org/citation.cfm?id=1553439) In Proceedings of the 26th Annual International Conference on Machine Learning, pp. 497-504. ACM, 2009.
9. Rosenblatt, Frank. ["The perceptron: A probabilistic model for information storage and organization in the brain."](http://psycnet.apa.org/journals/rev/65/6/386/) Psychological review 65, no. 6 (1958): 386.
10. Bishop, Christopher M. ["Pattern recognition."](http://users.isr.ist.utl.pt/~wurmd/Livros/school/Bishop%20-%20Pattern%20Recognition%20And%20Machine%20Learning%20-%20Springer%20%202006.pdf) Machine Learning 128 (2006): 1-58.

